# Example 4: Securitization
This notebook demonstrates how securitization contributes to the desctruction of total money stocks.

We start by importing required utilities. Technical notes: the code assumes that the notebook is saved in the money creation examples folder, if you wish to save it in a different folder modify the path below accordingly (if you wish to execute or modify the code herein, it can be viewed without that). Furthermore, the cells containing booking statements should only be executed once, because the booking statements will be recorded on each execution, which may result in assertion errors, e.g. when more cash is deducted than the agent possesses. If you wish to execute the booking statements again, then all cells starting from the declaration of the agents should be executed again, in order to reset the system and repeat the sequence of booking statements.

In [1]:
import os
import sys

base_path = os.path.realpath(os.getcwd()+"/../..")
sys.path.append(base_path)

from abcFinance import Ledger, Account, AccountSide

## Declaration of agents
Our population of agents in this examples exists of one bank and two different types of private sector agent: one household and one investment fund.

In [2]:
bank = Ledger(residual_account_name="Equity")
household = Ledger(residual_account_name="Equity")
fund = Ledger(residual_account_name="Equity")

bank.make_asset_accounts(['Reserves','Loans','Asset-backed securities'])
bank.make_liability_accounts(['Wholesale Deposits','Retail Deposits'])

fund.make_asset_accounts(['Deposits','Asset-backed securities'])

household.make_asset_accounts(['Deposits'])
household.make_liability_accounts(['Loans'])

We define a function that computes the money stocks according to our defined taxonomy:

In [3]:
def print_money_stocks():
    # Bank money: bank liabilities that are money
    bank_money = bank.get_balance('Wholesale Deposits')[1]
    bank_money += bank.get_balance('Retail Deposits')[1]
        
    print("Total (Bank) Money:",bank_money)

def print_balance_sheets_and_money_stocks():
    print("Bank balance sheet:")
    bank.print_balance_sheet()
    print("Household balance sheet:")
    household.print_balance_sheet()
    print("Fund balance sheet:")
    fund.print_balance_sheet()
    print_money_stocks()

## Start of the example
We initialize the system with a simple set of endowments: the fund owns some money holdings, the household's money holdings come entirely from a bank loan, and the bank's balance sheet only reflects these two contracts.

In [4]:
bank.book(debit=[('Reserves',100),('Loans',100)],credit=[('Wholesale Deposits',100),('Retail Deposits',100)])
fund.book(debit=[('Deposits',100)],credit=[('Equity',100)])
household.book(debit=[('Deposits',100)],credit=[('Loans',100)])
print_balance_sheets_and_money_stocks()

Bank balance sheet:
Asset accounts:
   Reserves : 100
   Loans : 100
Liability accounts:
   Wholesale Deposits : 100
   Retail Deposits : 100
Equity:  0
Total Assets:  200
--
Household balance sheet:
Asset accounts:
   Deposits : 100
Liability accounts:
   Loans : 100
Equity:  0
Total Assets:  100
--
Fund balance sheet:
Asset accounts:
   Deposits : 100
Liability accounts:
Equity:  100
Total Assets:  100
--
Total (Bank) Money: 200


Now the bank securitizes the household's loan, leaving total money stocks unaffected:

In [5]:
bank.book(debit=[('Asset-backed securities',100)],credit=[('Loans',100)])
bank.print_balance_sheet()
print_money_stocks()

Asset accounts:
   Reserves : 100
   Asset-backed securities : 100
Liability accounts:
   Wholesale Deposits : 100
   Retail Deposits : 100
Equity:  0
Total Assets:  200
--
Total (Bank) Money: 200


Now the fund purchases the securitized loans, destroying the money corresponding to the purchase price:

In [6]:
bank.book(debit=[('Wholesale Deposits',100)],credit=[('Asset-backed securities',100)])
fund.book(debit=[('Asset-backed securities',100)],credit=[('Deposits',100)])
print_balance_sheets_and_money_stocks()

Bank balance sheet:
Asset accounts:
   Reserves : 100
Liability accounts:
   Retail Deposits : 100
Equity:  0
Total Assets:  100
--
Household balance sheet:
Asset accounts:
   Deposits : 100
Liability accounts:
   Loans : 100
Equity:  0
Total Assets:  100
--
Fund balance sheet:
Asset accounts:
   Asset-backed securities : 100
Liability accounts:
Equity:  100
Total Assets:  100
--
Total (Bank) Money: 100
